In [1]:
from docx import Document
import csv
from tensorflow.keras.preprocessing.text import text_to_word_sequence
import numpy as np
from numpy import dot
from numpy.linalg import norm
import numpy as np
import re
from nltk.corpus import stopwords
import nltk
from nltk.stem import PorterStemmer, LancasterStemmer
import os
from tqdm import tqdm
from tqdm import trange



In [2]:
nltk.download('stopwords')
nltk.download('tagsets')
nltk.download('averaged_perceptron_tagger')

from nltk.tag import pos_tag

from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KJH\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\KJH\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\KJH\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\KJH\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:

# 본문 텍스트 추출
def parsing(par1):
    last = []
    l_style =[]
    i = 0
    c = 0
    text = ""

    #p 속성 파싱
    for a in par1:
        p_style = a.style.name
        p_bold = a.style.font.bold
        p_font = a.style.font.name
        p_size = a.style.font.size
        if p_size != None:
            p_size = p_size.pt   
        p_italic = a.style.font.italic
        p_underline = a.style.font.underline

        #r속성 파싱
        for test in a.runs:
            if (test.text.isspace() == False) and (test.text != ""):
                last.append([])

                #소문자 변경
                text = test.text.lower()

                #한글/영어 제외 제거
                text = re.compile('[^ ㄱ-ㅣ가-힣|a-z]+').sub('', text)

                last[i].append(text)


                #글자 속성 파싱(굵게)
                if test.bold == None:
                    if test.style.name == "Default Paragraph Font":
                        last[i].append(p_bold)
                    else:
                        last[i].append(test.style.font.bold)
                else:
                    last[i].append(test.bold)


                #글자 속성 파싱(글꼴)
                if test.font.name == None:
                    last[i].append(p_font)
                else:
                    last[i].append(test.font.name)

                #글자 속성 파싱(글자 크기)
                if test.font.size == None:
                    if test.style.name == "Default Paragraph Font":
                        if p_size != None:
                            last[i].append(p_size)
                        else:
                            last[i].append(float(10))
                    elif test.style.font.size != None:
                        last[i].append(test.style.font.size.pt)
                    else:
                        last[i].append(float(10))

                else:
                    last[i].append(test.font.size.pt)

                #글자 속성 파싱(기울임 꼴)
                if test.italic == None:
                    if test.style.name == "Default Paragraph Font":
                        last[i].append(p_italic)
                    else:
                        last[i].append(test.style.font.italic)
                else:
                    last[i].append(test.italic)


                #글자 속성 파싱(밑줄)
                if test.underline == None:
                    if test.style.name == "Default Paragraph Font":
                        last[i].append(p_underline)
                    else:
                        last[i].append(test.style.font.underline)
                else:
                    last[i].append(test.underline)

                #글자 속성 파싱(스타일)
                if test.style.name == "Default Paragraph Font":
                    last[i].append(p_style)
                    l_style.append(p_style)
                else:
                    last[i].append(test.style.name)
                    l_style.append(test.style.name)

                c = i
                i= i+1

        if len(a.runs)!= 0 and (text.isspace() == False) and (text != ""):
            last[c][0] = last[c][0] +" "
    


    
    return last

In [4]:
#노이즈 제거 (문서 작성 시 사용되는 다중 띄어쓰기 등)
def Dedupli(last):
    i = 0
    j = 0



    while i < len(last)-1:
        if i + j >= len(last)-1:
            break

        for j in range(1, len(last)-i):
            if last[i][1] == last[i+j][1] and last[i][2] == last[i+j][2] and last[i][3] == last[i+j][3] and last[i][4] == last[i+j][4] and last[i][5] == last[i+j][5] and last[i][6] == last[i+j][6]:
                last[i][0] = last[i][0] + last[i+j][0]
                last[i+j][0] =""
            else:
                i = i+j
                break

    
    real_last = [[]]

    for text in last[:]:
        a = text[0].isspace()
        if (a==True) or (text[0] == ""):
            del text
        elif (len(text[0]) == 1):
            del text
        else:
            real_last.append(text)


    return real_last

In [5]:
#전처리(영어 어간 추출)
def pret(texts):
    stops = set(stopwords.words('english'))
    no_stops = []
    a = []
    
    l = len(texts)
    i = 1
    c = 0
    stemmer = PorterStemmer()
    text_en = []



    for j in range(len(texts)-1):
        a = text_to_word_sequence(texts[i][0])
        for t in a:
            if t not in stops:
                no_stops.append(t)

        #text_en = arche_type(no_stops)
        for c in range(0,len(no_stops)):
            text_en.append(stemmer.stem(no_stops[c]))
            



        texts[i][0] = " ".join(map(str, text_en))
        i = i+1
        no_stops = []
        text_en = []


    return texts

In [6]:
#CSV 용 추가사항
def pret_plus(real_last):
    for i in range(1, len(real_last)):
            if real_last[i][1] != True:
                real_last[i][1] = "None"

            if real_last[i][4] != True:
                real_last[i][4] = "None"

            if real_last[i][5] != True:
                real_last[i][5] = "None"

            if real_last[i][2] == None:
                real_last[i][2] = "기본 글꼴"
    return real_last


In [7]:
#품사 태깅 및 매칭
def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('V'):
        return 'v'
    if pos_tag.startswith('N'):
        return 'n'
    elif pos_tag.startswith('J'):
        return 'a'
    elif pos_tag.startswith('R'):
        return 'r'
    else:
        return None

In [8]:
#텍스트 원형 복원
def arche_type(test_text):

    tokens_pos_list = pos_tag(test_text)



    tokens_pos_list2 = []
    for token_list in tokens_pos_list:
        temp_list = []
        (token, pos_tags) = token_list
        tag = get_wordnet_pos(pos_tags)
        if tag!=None:
            temp_list.append((token, get_wordnet_pos(pos_tags)))
        tokens_pos_list2.append(temp_list)


    lemm = WordNetLemmatizer()
    token_final = []
    for token_pos in tokens_pos_list2:
        for token, tag in token_pos:
            token_final.append(lemm.lemmatize(token, pos=tag))

    return token_final


In [9]:
#가중치 함수
def weighting(real_last):
    all_len = 0
    n_bol_len =0
    bol_len = 0
    n_inc_len =0
    inc_len =0
    size_len =[]
    n_und_len =0
    und_len =0

    #토큰수에 따른 비율 설정
    for i in range(1, len(real_last)):
        tok_len = len(text_to_word_sequence(real_last[i][0]))
        real_last[i].append(tok_len)

        if real_last[i][3] != None:
            for j in range(tok_len):
                size_len.append(int(real_last[i][3]))
        else:
            for j in range(tok_len):
                size_len.append(int(10))

        if real_last[i][4] == True:
            inc_len = inc_len + tok_len
        else:
            n_inc_len = n_inc_len + tok_len

        if real_last[i][5] == True:
            und_len = und_len + tok_len
        else:
            n_und_len = n_und_len + tok_len

        if real_last[i][1] == True:
            bol_len = bol_len + tok_len
        else:
            n_bol_len = n_bol_len + tok_len


        all_len = all_len + tok_len

    std = np.std(size_len)
    mean = np.mean(size_len)




    #가중치 설정
    for i in range(1, len(real_last)):
        #기본값
        a = 0.2
        if real_last[i][1] == True:
            a = a+(n_bol_len/all_len)
        else:
            a = a+(bol_len/all_len)
        
        if std != 0:
            w = float((real_last[i][3]-mean)/mean)
            a = a+w
        else:
            pass

        if real_last[i][4] == True:
            a = a+(n_inc_len/all_len)
        else:
            a = a+(inc_len/all_len)

        if real_last[i][5] == True:
            a = a+(n_und_len/all_len)
        else:
            a = a+(und_len/all_len)


        real_last[i].append(a)




    return real_last



In [10]:
#코사인 값 함수
def cos_sim(a, b):
    return dot(a, b)/(norm(a)*norm(b))

#가중치 추가한 빈도수 함수
def make_matrix(feats, list_data , we):
    freq_list = []
    data = []
    tok_we = []
    j = 0
    for text in list_data:
        tok = text_to_word_sequence(text)
        data = data+tok        
        for i in range(len(tok)):
            tok_we.append(we[j])
        j +=1

    for feat in feats:
        freq = 0
        t = 0
        for word in data:
            if feat == word:
                freq += tok_we[t]
            t+=1
        freq_list.append(freq)


    return freq_list

#기존 코사인 알고리즘을 위한 빈도 행렬 함수
def make_matrix2(feats, list_data):
    freq_list = []
    for feat in feats:
        freq = 0
        for word in list_data:
            if feat == word:
                freq += 1
        freq_list.append(freq)
    return freq_list


In [11]:
#본문 텍스트 및 가중치
def dataization(real_last):
    i = 1
    j = 1
    test1 = []
    test1.append([])
    test1.append([])

    for j in range(len(real_last)-1):
        if len(real_last[i][0]) > 1 :
            test1[0].append(real_last[i][0])
            test1[1].append(real_last[i][8])
            i += 1

    return test1

In [30]:
#파싱 및 전처리 과정 합친 함수(실험 1, 문서 분할)
def all_parsing(d):
    doc = Document(d)
    Num_par = len(doc.paragraphs)
    par1 = doc.paragraphs

    test = parsing(par1)
    test_last = Dedupli(test)    
    test_last = pret(test_last)   
    test_last = weighting(test_last)


    test_last = dataization(test_last)
    
    first_array = []
    first_array.append([])
    first_array.append([])
    second_array = []
    second_array.append([])
    second_array.append([])

    e = 0

    for e in range(len(test_last[0])):
        if e%4 == 3:
            first_array[0].append(test_last[0][e])
            first_array[1].append(test_last[1][e])

        else:
            second_array[0].append(test_last[0][e])
            second_array[1].append(test_last[1][e])

    return first_array, second_array
    

In [13]:
#파싱 및 전처리(동일서식 확인)
def all_parsing2(d):
    doc = Document(d)
    par1 = doc.paragraphs

    test = parsing(par1)
    test_last = Dedupli(test)    
    test_last = pret(test_last)   
    test_last = weighting(test_last)


    test_last = dataization(test_last)
    
    return test_last
    

In [14]:
#실험 유사도 측정 함수
def test_fuction(path):
    a,b = all_parsing(path)


    z1 = " ".join(s for s in a[0])
    z2 = " ".join(s for s in b[0])


    t1 = text_to_word_sequence(z1)
    t2 = text_to_word_sequence(z2)


    feat2 = set(t1+t2)

    #가중치를 추가한 각 빈도


    v1 = np.array(make_matrix(feat2, a[0], a[1]))
    v2 = np.array(make_matrix(feat2, b[0], b[1]))

    cs = cos_sim(v1, v2)


    feats2 = set(t1+ t2)


    v3 = np.array(make_matrix2(feats2, t1))
    v4 = np.array(make_matrix2(feats2, t2))


    cs1 = cos_sim(v3, v4)

    print("name : " + str(path) + " 가중치 : " +str(cs) +" 기존 : " + str(cs1))

    return cs, cs1


In [31]:
#가중치 및 기존함수 비교

path = 'C:/Users/KJH/Desktop/last_folder/test2'

e = []
f = []

files = os.listdir(path)

for i in range(len(files)):
    files[i] = 'C:/Users/KJH/Desktop/last_folder/test2/' + files[i]


for i in tqdm(range(len(files))):
    c,d = test_fuction(files[i])
    e.append(c)
    f.append(d)


print("가중치 평균" +str(sum(e)/len(e)))
print("기존 평균" + str(sum(f)/len(f)))

  7%|▋         | 1/15 [00:02<00:28,  2.07s/it]

name : C:/Users/KJH/Desktop/last_folder/test2/119.docx 가중치 : 0.7896690344803345 기존 : 0.766463601704344


 13%|█▎        | 2/15 [00:05<00:35,  2.73s/it]

name : C:/Users/KJH/Desktop/last_folder/test2/14.docx 가중치 : 0.8983551415662834 기존 : 0.8849295319684343


 20%|██        | 3/15 [00:06<00:25,  2.10s/it]

...

name : C:/Users/KJH/Desktop/last_folder/test2/dataset[learn]_33.docx 가중치 : 0.8565402819146628 기존 : 0.8563125583741192
name : C:/Users/KJH/Desktop/last_folder/test2/dataset[learn]_353.docx 가중치 : 0.6339919326124303 기존 : 0.6429788582206545


 93%|█████████▎| 14/15 [00:28<00:01,  1.55s/it]

name : C:/Users/KJH/Desktop/last_folder/test2/dataset[learn]_411.docx 가중치 : 0.687493299139179 기존 : 0.6130911208226802


100%|██████████| 15/15 [00:31<00:00,  2.08s/it]

name : C:/Users/KJH/Desktop/last_folder/test2/dataset[learn]_723.docx 가중치 : 0.8210409788652903 기존 : 0.826251015043785
가중치 평균0.7455293907713987
기존 평균0.7129036058477721


In [46]:
#실험 2
doc1 = 'C:/Users/KJH/Desktop/docx테스트/최종데이터셋/1/테스트4.docx'
doc2 = 'C:/Users/KJH/Desktop/docx테스트/최종데이터셋/1/테스트원본.docx'

c = all_parsing2(doc1)
d = all_parsing2(doc2)


z1 = " ".join(s for s in c[0])
z2 = " ".join(s for s in d[0])


t1 = text_to_word_sequence(z1)
t2 = text_to_word_sequence(z2)


feat2 = set(t1+t2)

#가중치를 추가한 각 빈도


v1 = np.array(make_matrix(feat2, c[0], c[1]))
v2 = np.array(make_matrix(feat2, d[0], d[1]))

cs = cos_sim(v1, v2)


feats2 = set(t1+ t2)


v3 = np.array(make_matrix2(feats2, t1))
v4 = np.array(make_matrix2(feats2, t2))


cs1 = cos_sim(v3, v4)

print("name :  가중치 : " +str(cs) +" 기존 : " + str(cs1))

name :  가중치 : 0.9999830884367636 기존 : 0.9999999999999998


In [17]:
#확인용 csv 파일 생성
fields = ['Text', 'Bold', 'Font', 'Size', 'Italic', 'Underline','Style','S_Bold', 'S_Size', 'S_Italic', 'S_Underline']

d = "C:/Users/KJH/Desktop/last_folder/test3/Community.docx"


doc = Document(d)
Num_par = len(doc.paragraphs)
par1 = doc.paragraphs

test = parsing(par1)
test_last = Dedupli(test)    
test_last = pret(test_last)   
test_last = pret_plus(test_last)



with open('case1.csv','w',newline='',encoding='utf-8-sig') as file :

    write = csv.writer(file)
    write.writerow(fields)
    

    for i in range(len(test_last)):
        write.writerow(test_last[i])
